# Transfer Learning on Oyster and green oyster mushroom

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
from tensorflow import keras
from keras.applications.mobilenet import MobileNet, decode_predictions, preprocess_input
from keras import preprocessing
from tensorflow.keras.preprocessing import image
import keras.backend as K
from keras.layers import Dense,Flatten,GlobalAveragePooling2D,InputLayer,Dropout

### Pretrained Model: MobileNet from Keras (https://keras.io/api/applications/)

In [ ]:
img = image.load_img('oyster.png')
img

In [ ]:
type(img)

##### Expand dimensions

In [ ]:
x = np.array(img)
x.shape 

In [ ]:
X = np.array([x]) # Hint: Model expects a batch 
X.shape           # i.e. a "batch" of one image, with the rest of expected shape...reminiscient of the data types expected by sklearn models

#### Prepare data based on specifications of pre-trained model

In [ ]:
X = preprocess_input(X) # essentially: scale vals between 0 and 255 to -1 < x < 1

#### Load MobileNet model

In [ ]:
model = MobileNet()  # download (on `ImageNet` pretrained) model 

#### Test model

In [ ]:
pred = model.predict(X)
pred.shape

In [ ]:
pred[0,:10] # MobileNet predicts whether an image belongs to one of 1k classes (i.e. prob of belonging to one of these classes)

In [ ]:
decode_predictions(pred) # make these preds "human-readable"

#### Check what is really in the image

In [ ]:
plt.imshow(img);

In [ ]:
model.summary()

# Transfer Learning for Neural Networks

In [ ]:
# folder names containing images of the things you want to classify
classes = ['oyster','green_oyster']
# plug in the path to your data folder
base_path = './images/folder/'

In [ ]:
# define the preprocessing function that should be applied to all images
data_gen = preprocessing.image.ImageDataGenerator(   # loads data in batches from disk
    preprocessing_function=preprocess_input,
    # fill_mode='nearest',
    rotation_range=20,                               # rotate image by a random degree between -20 and 20
    # width_shift_range=0.2,                         # shift image horizontally 
    # height_shift_range=0.2,                        # shift image vertically 
    # horizontal_flip=True,                          # randomly flip image horizontally
    zoom_range=0.5,                                  # apply zoom transformation using zoom factor between 0.5 and 1.5
    # shear_range=0.2                                # shear rotates pics, but makes them be in trapezoids (as opposed to squares)
    validation_split=0.2
)

In [ ]:
# a generator that returns batches of X and y arrays
train_data_gen = data_gen.flow_from_directory(      # points to dir where data lives
        directory=base_path,
        class_mode="categorical",
        classes=classes,
        batch_size=32,
        target_size=(224, 224),
    subset='training'
)

In [ ]:
val_data_gen = data_gen.flow_from_directory(
        directory=base_path,
        class_mode="categorical",
        classes=classes,
        batch_size=32,
        target_size=(224, 224),
    subset='validation'
)

In [ ]:
train_data_gen.class_indices

In [ ]:
classes

## Create CNN Model as Base Model

### 1. Select the convolutional base 

#### Using MobileNet pretrained network for transfer learning

In [ ]:
K.clear_session()
base_model = MobileNet(
    weights='imagenet',
    include_top=False,                          # keep convolutional layers only
    input_shape=(224, 224, 3)
)

In [ ]:
base_model.summary()                            # as expected...see above

### 2. Freeze the weights

In [ ]:
base_model.trainable = False  # we don't want to train the base model, since this would destroy filters

## Build your "individualized" architecture for "top-layers"

### 3. Add your own dense layers on top

In [ ]:
len(classes)

In [ ]:
model = keras.Sequential()
model.add(base_model)
model.add(Flatten())  
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(len(classes), activation='softmax')) # TODO; Final layer with a length of 2, and softmax activation

In [ ]:
model.summary()                 # Note "non-trainable" params...

### 4. Compile and train

In [ ]:
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),
              loss=keras.losses.categorical_crossentropy, #TODO: why not binary x-entropy?
              metrics='accuracy')

# observe the validation loss and stop when it does not improve after 3 iterations
callback = keras.callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0.05,     # the minimum expected change in the metric used in order to be seen as an improvement
    patience=3,         # number of epochs with no improvement needed for the model to stop
    restore_best_weights=True,
    mode='min'
    )

#### Hint: Make sure, you have enough memory (RAM) on your computer. Instead, you could use Google Colab or Kaggle to run the whole script + data on the GPU ("Laufzeittyp ändern -> Hardwarebeschleuniger GPU")

In [ ]:
history = model.fit(train_data_gen,
          verbose=2, 
          callbacks=[callback],
          epochs=20,
          validation_data=val_data_gen
          )

In [ ]:
plt.plot(history.history['loss'], label = 'Training Loss')
plt.plot(history.history['val_loss'], label = 'Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend();

### (5. Use it to predict)

In [ ]:
img = image.load_img('object.png',target_size=(224,224))

In [ ]:
plt.imshow(img);

In [ ]:
img.size

In [ ]:
x = np.array(img)

In [ ]:
X = np.array([x]) 
X.shape 

In [ ]:
X_preprocess = preprocess_input(X)

In [ ]:
pred = model.predict(X_preprocess)
pred

In [ ]:
plt.bar(x = classes, height = pred[0]);

### (6. Save your model for later)

In [ ]:
# model.save('models/wallet_phone.h5')